In [2]:
# Fasttext main branch has issue in calculation of F1 score. hence using this label from github
!pip install git+https://github.com/facebookresearch/fastText.git@b64e359d5485dda4b4b5074494155d18e25c8d13 -q

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')

In [4]:
from prepare_ft_file import *
import pandas as pd
from sklearn.model_selection import train_test_split
import fasttext

In [5]:
# Load MP3 mapped data
data = pd.read_json('/content/drive/MyDrive/annamayya_dataset_cleaned.json')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['Lyric'], data['genre'], random_state=42, test_size=0.1)
# Do another split on train data for validation purpose
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=42, test_size=0.3)

In [7]:
write_ft_file(X_train, y_train, 'train.txt')
write_ft_file(X_valid, y_valid, 'valid.txt')
write_ft_file(X_test, y_test, 'test.txt')

In [8]:
# Auto tune Fast Text Model
ft_auto_tune_model = fasttext.train_supervised(input="train.txt", 
                                    autotuneValidationFile='valid.txt', 
                                    autotuneModelSize="500k", 
                                    autotuneMetric='f1',
                                    dim=200)

In [10]:
def get_results(file, model, X):
  r = model.test_label(file)
  precision = (r['__label__Devotional']['precision'] + r['__label__Romantic']['precision'])/2
  recall = (r['__label__Devotional']['recall'] + r['__label__Romantic']['recall'])/2
  f1 = 2*precision*recall/(precision+recall)
  return [len(X), f1,precision,recall]
results = pd.DataFrame()
results.loc['Train',['Samples','F1','Precision','Recall']] = get_results('train.txt', ft_auto_tune_model, X_train)
results.loc['Validation',['Samples','F1','Precision','Recall']] = get_results('valid.txt', ft_auto_tune_model, X_valid)
results.loc['Test',['Samples','F1','Precision','Recall']] = get_results('test.txt', ft_auto_tune_model, X_test)
results

,Samples,F1,Precision,Recall
Train,8994.0,0.999821,0.999931,0.999711
Validation,3855.0,0.918640,0.938528,0.899577
Test,1428.0,0.906173,0.918555,0.894121
